# Experiments implementation
Implementing the convolutional neural networks models and experiments implementations

In [1]:
#imports
import numpy as np
import sklearn

global data_folder,dataset_file
data_folder = "../data/"
dataset_file= ["cullpdb+profile_6133.npy.gz","cullpdb+profile_5926_filtered.npy.gz","CB513.npy"]

### Data loading procedures
#### Explanation
Dataset split
int the dataset cullpdb+profile_5926.npy.gz
the subset  division of the data set is as follows:
[0,5430) training
[5435,5690) test
[5690,5926) validation



The subset division of the cullpdb+profile_6133.npy.gz is as follows:
The dataset division for the first cullpdb+profile_6133.npy.gz dataset is
[0,5600) training
[5605,5877) test
[5877,6133) validation


In both datasets cullpdb+profile_5926.npy.gz and cullpdb+profile_6133.npy.gz
The features in the dataset can be found as follows:

It is currently in numpy format as a (N protein x k features) matrix.
You can reshape it to (N protein x 700 amino acids x 57 features) first.

The 57 features are:
[0,22): amino acid residues, with the order of 'A', 'C', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'M', 'L', 'N', 'Q', 'P',
                                               'S', 'R', 'T', 'W', 'V', 'Y', 'X','NoSeq'
[22,31): Secondary structure labels, with the sequence of 'L', 'B', 'E', 'G', 'I', 'H', 'S', 'T','NoSeq'
[31,33): N- and C- terminals;
[33,35): relative and absolute solvent accessibility, used only for training.
        (absolute accessibility is thresholded at 15; relative accessibility
        is normalized by the largest accessibility value in a protein and thresholded at 0.15;
        original solvent accessibility is computed by DSSP)
[35,57): sequence profile. Note the order of amino acid residues is ACDEFGHIKLMNPQRSTVWXY and
         it is different from the order for amino acid residues

The last feature of both amino acid residues and secondary structure labels just mark end of the protein sequence.
[22,31) and [33,35) are hidden during testing.

source: https://www.princeton.edu/~jzthree/datasets/ICML2014/dataset_readme.txt
accessed in 29/06/2019

In [2]:
def buildScoringMatrix():
    """
    Building protein scoring matrix and creating a dictionary with the aminoacids and each of their corresponding
    rows in the matrix
    To convert the one hot encoding of the aminoacids for simplicity it can be made a dot product with the matrix
    M with the onehot encoding of the aminoacid.
    :return: Scoring Matrix M, positions of the aminoacids 
    """
    positions = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,
                 'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}
    M=[[4,   0, -1, -2,  0, -1, -2, -3, -1, -1,  1,  0, -2, -2, -1, -2, -1, -2, -2, -1],
       [-1, -2, -3 ,-3 ,-3 ,-3, -4, -4,  8, -3, -1, -1, -3, -2, -1, -2, -1, -2, -2, -1],
       [3,   2, -2 ,-2 ,-1 ,-1, -3, -3, -1, -1,  1,  3, -2, -1, -1, -2, -1, -2, -2, -1],
       [2,  -2,  1,  0,  2,  0,  3, -2, -2, -1,  0, -1,  0, -2, -2, -2, -2, -2, -3, -2],
       [0,  -1, -1, -2, -1, -1, -2, -3, -1, -1,  3,  4, -2,  0, -1, -2, -1, -1, -1, -1],
       [0,  -3,  3,  1,  4,  1, -1, -3, -3, -1, -2,  0, -1, -3, -2, -3, -3, -3, -3, -3],
       [0,  -1, -1, -2, -1, -1, -2, -3, -1, -1, -3,  4, -2,  0, -1, -2, -1, -1, -1, -1],
       [-1, -2, -3, -3, -3, -3, -4, -4,  8, -3, -1, -1, -3, -2, -2, -2, -1, -2, -2, -1],
       [3,   0, -2, -2, -1, -1, -2, -3, -1, -1,  3,  1, -2, -1, -1, -1, -1, -1, -1, -1],
       [1,  -1 ,-2 ,-2 ,-1 ,-1, -3, -3, -1, -1,  4,  3, -2,  0,  0, -1, -1, -1, -1, -1],
       [0,   6, -4, -4, -3 ,-3, -3, -3, -2, -3,  0, -2, -3, -1, -2, -2, -2, -3, -2, -2],
       [0,  -3 , 1,  4,  1 , 2,  0, -2, -3, -1, -2, -1, -1, -3, -2, -3, -2, -2, -4, -3],
       [2,   0 ,-2, -2, -1, -2, -3, -3, -1, -1,  4,  1, -2,  0,  0, -1,  0, -1, -1,  0],
       [-2, -1 ,-3 ,-4 ,-3, -3, -4, -4, -2, -4,  0, -1, -3,  3,  0, -1, -1, -2,  6,  1],
       [0 ,  6, -4, -4, -3, -3, -3, -3, -2, -3,  0, -2, -3, -1, -2, -2, -2, -3, -2, -2],
       [1 , -1 ,-2 ,-2, -1, -1, -3, -2, -1, -2,  1,  3, -2,  0,  4, -1,  0,  0, -1,  0],
       [0,  -2,  1, -1,  2,  0, -2, -3, -3, -1,  2,  3, -1, -1, -1, -2, -1, -2, -2, -1],
       [0,  -3,  3,  1,  4,  1, -1, -3, -3, -1, -2,  0, -1, -3, -2, -3, -3, -3, -3, -3],
       [0 , -1 ,-2, -2, -1, -1, -3, -3, -1, -2,  2,  3, -2,  0,  0, -1,  3,  1, -1,  0],
       [0,  -3,  3,  1,  4,  1, -1, -3, -3, -1, -2,  0, -1, -3, -2, -3, -3, -3, -3, -3],
       [3,   0, -2, -2, -1, -1, -3, -3, -1, -1,  3,  1, -2,  0,  0, -1, -1, -1, -1, -1]]
    return M, positions

In [3]:
def load_and_reshape(ds_idx=0):
    global data_folder, dataset_file
    ds = np.load(data_folder+dataset_file[ds_idx])
    print("[INFO] The data set "+dataset_file[ds_idx]+"  has been loaded")
    ds = ds.reshape(-1,700,57)
    return ds

In [5]:
def extract_labels_from_samples(data):
    """
    As indicated in the description above, the labels are then extracted from the
    fetures loaded with the data and return with the data reshaped
    note that the N- and C- terminals are being ignored as well as the relative and absolute solvent accessibility
    and the sequence profile.
    :param data: The data loaded
    :return: data, labels
    """
    labels_start = 22
    labels_end = 31
    aminoacids_end = 22

    data_samples = data[:,:,:aminoacids_end]
    labels  =data[:,:,labels_start:labels_end]
    labels  = np.array([np.argmax(labels[i,:]) for i in range(labels.shape[0])])
    return data_samples,labels

In [6]:
def use_scoring_matrix(data):
    """
        Using scoring matrix defined in buildScoringMatrix in order to replace the one hot enconding
        of the aminoacid with the encoding defined in the scoring matrix.
        return: data set with the one hot encoded samples replaced by their corespondin position in the scoring matrix
    """
    M, _ = buildScoringMatrix()
    newData = [[]]
    for i,d in enumerate(data):
        for aminoacid in d:
            aminoacid = np.dot(aminoacid,M).tolist()
            #Replacing one hot enconding with aminoacid encoding from the scoring matrix
            newData[i].append(aminoacid)
        newData.append([])
    return np.array(newData)

In [7]:
def load_and_split(ds_opc):
    """
    :param ds_opc: data set that has been selected
           The ds_opc must indicate an option choice between the following sets
           CB513.npy,cullpdb+profile_6133.npy.gz,cullpdb+profile_5926_filtered.npy.gz
    :return:  train, test and validation sets (Already shuffled)
    """
    assert type(ds_opc) == int
    assert ds_opc < 3 and ds_opc >= 0
    print("[INFO] The dataset " + dataset_file[ds_opc] + "Has been selected")

    data = load_and_reshape(ds_opc)
    # Shuffling the data
    data = sklearn.utils.shuffle(data)

    data, labels = extract_labels_from_samples(data)

    train_set = None
    train_labels = None

    test_set = None
    test_labels = None

    valid_set = None
    valid_labels = None

    if ds_opc == 0:
        # spliting the cullpdb+profile_6133.npy.gz
        train_idx = [0, 5430]
        test_idx = [5435, 5690]
        valid_idx = [5690, 5926]

        train_set = data[:train_idx[1], :, :]
        train_labels = labels[: train_idx[1]]

        test_set = data[test_idx[0]:test_idx[1], :, :]
        test_labels = labels[test_idx[0]:test_idx[1]]

        valid_set = data[valid_idx[0]:valid_idx[1], :, :]
        valid_labels = labels[valid_idx[0]:valid_idx[1]]

    elif ds_opc == 1:
        # spliting the cullpdb+profile_5926_filtered.npy.gz
        train_idx = [0, 5600]
        test_idx = [5605, 5877]
        valid_idx = [5877, 6133]

        train_set = data[:train_idx[1], :, :]
        train_labels =labels[ :train_idx[1]]

        test_set = data[test_idx[0]:test_idx[1], :, :]
        test_labels = labels[test_idx[0]:test_idx[1]]

        valid_set = data[valid_idx[0]:valid_idx[1], :, :]
        valid_labels = labels[valid_idx[0]:valid_idx[1]]

    elif ds_opc == 2:
        # spliting CDB513
        n_samples = 513
        # Spliting the dataset in 70% for train, 20% for test and 10% for validation
        train_val = int(n_samples * 0.7)
        test_val = train_val + int(n_samples * 0.2)
        valid_val = test_val + int(n_samples * 0.1)
        train_idx = [0, train_val]
        test_idx = [train_val, test_val]
        valid_idx = [test_val, valid_val]

        train_set = data[:train_idx[1], :, :]
        train_labels = labels[: train_idx[1]]

        test_set = data[test_idx[0]:test_idx[1], :, :]
        test_labels = labels[test_idx[0]:test_idx[1]]

        valid_set = data[valid_idx[0]:valid_idx[1], :, :]
        valid_labels = labels[valid_idx[0]:valid_idx[1]]

    return train_set,train_labels, test_set,test_labels, valid_set,valid_labels

### Models Implementation

In [8]:
train_set,train_labels,test_set,test_labels,valid_set,valid_labels = load_and_split(2)

[INFO] The dataset CB513.npyHas been selected
[INFO] The data set CB513.npy  has been loaded
